In [1]:
# installing wandb
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.8 MB/s eta 0:00:00


In [4]:
# initiating wandb
import wandb
wandb.login()
wandb.init(
    project =  "MLP on MNIST",
    config = {
        "epochs":25,
        "batch_size":100,
        "lr":0.001
    }
)
config = wandb.config

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [18]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [6]:
# network architecture variables

input_size = 784 # number of input neurons (28 x2 28)
hidden_layer_size = 400 # (784+10)/2
output_size = 10

In [8]:
# import MNIST from trochvision

train_dataset = datasets.MNIST(
    root = './data',
    train=True,
    transform=transforms.ToTensor(),
    download = True
)


test_dataset = datasets.MNIST(
    root = './data',
    train=False,
    transform=transforms.ToTensor(),
    download = True
)



In [12]:
type(train_dataset)

torchvision.datasets.mnist.MNIST

In [15]:
# create tarining and testing data batches

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=config.batch_size,
    shuffle = True
)


test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=config.batch_size,
    shuffle = True
)

In [31]:
# define our network architechture

class MNISTNet(nn.Module):

    def __init__(self,input_size, hidden_layer_size, output_size ):
      super(MNISTNet,self).__init__()
      self.fc1 = nn.Linear(input_size,hidden_layer_size)
      self.fc2 = nn.Linear(hidden_layer_size, hidden_layer_size)
      self.fc3 = nn.Linear(hidden_layer_size, output_size)
      self.relu = nn.ReLU()
      # new addition -> define initiaol weights
      self.init_weights()

    # Weight initilization Functioin
    def init_weights(self):
      nn.init.kaiming_normal_(self.fc1.weight)
      nn.init.kaiming_normal_(self.fc2.weight)


    def forward(self,x):
      out =self.fc1(x)
      out = self.relu(out)
      out =self.fc2(out)
      out = self.relu(out)
      out =self.fc3(out)
      return out





In [32]:
# Preprare for training

network = MNISTNet(input_size,hidden_layer_size,output_size)

# Deploying our network in GPU
CUDA = torch.cuda.is_available()

if CUDA:
  network = network.cuda()

# loss function and optimizer

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(network.parameters(), lr=config.lr)


In [39]:
# training loop
for epoch in range(config.epochs):
  # correct training prediction
  correct_train = 0
  # loss for epoch
  updating_loss = 0

  for index, (images,lables) in enumerate(train_loader):
    # print(images.shape)
    images=images.view(-1,28*28)
    # print(images.shape)

    # deploying inputs into GPU
    if CUDA:
      imnages =images.CUDA()
    # feedforward

    outputs = network(images)

    # calculating the loss
    loss = loss_fn(outputs,lables)
    updating_loss += loss.item()

    #backpropergatrion
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # print(outputs.shape)

    _, predicted_index =torch.max(outputs.data,1)
    # print(predicted_index)
    correct_train += (predicted_index == lables).sum()

  params = {
      "Epoch": epoch+1,
      "Accuracy": (correct_train/len(train_loader)),
      "Loss": updating_loss/len(train_loader)
  }
  wandb.log(params)
  print(f"Epoch: {epoch+1}/{config.epochs}, Accuracy{(correct_train/len(train_loader))}, Loss{ updating_loss/len(train_loader)}")

wandb.finish()
print("Training is done")

Epoch: 1/25, Accuracy97.35832977294922, Loss0.08575785223705074
Epoch: 2/25, Accuracy98.3116683959961, Loss0.0540973250203145
Epoch: 3/25, Accuracy98.81500244140625, Loss0.03732735993495832
Epoch: 4/25, Accuracy99.09666442871094, Loss0.028724789929110556
Epoch: 5/25, Accuracy99.23500061035156, Loss0.022708206125438057
Epoch: 6/25, Accuracy99.34833526611328, Loss0.01965440045169089
Epoch: 7/25, Accuracy99.51499938964844, Loss0.014916488672351989
Epoch: 8/25, Accuracy99.48666381835938, Loss0.016127073539125073
Epoch: 9/25, Accuracy99.63500213623047, Loss0.010951292690612414
Epoch: 10/25, Accuracy99.54000091552734, Loss0.013597390659870144
Epoch: 11/25, Accuracy99.73999786376953, Loss0.007859065110860684
Epoch: 12/25, Accuracy99.66999816894531, Loss0.010111801418467318
Epoch: 13/25, Accuracy99.60832977294922, Loss0.013455175367419088
Epoch: 14/25, Accuracy99.6933364868164, Loss0.009056998004105784
Epoch: 15/25, Accuracy99.83833312988281, Loss0.004911492367858349
Epoch: 16/25, Accuracy99.8

Accuracy,▁▄▅▆▆▇▇▇▇▇█▇▇███▇████████
Epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
Loss,█▅▄▃▃▂▂▂▂▂▁▂▂▁▁▁▂▁▁▁▁▂▁▁▁
Accuracy,99.79166
Epoch,25
Loss,0.00675


Training is done
